In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import scikitplot

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

from modules import f1_metric

### **Read test data**

In [ ]:
df = pd.read_csv("E:/vindr_mammo/Data/full_df.csv")
df['image_path'] = 'E:/vindr_mammo/Data/png/' + df['study_id'] + '/' + df['image_id'] + '.png'
df['label'] = df['breast_birads'].map({'BI-RADS 1': '0',
                                       'BI-RADS 2': '0',
                                       'BI-RADS 3': '1',
                                       'BI-RADS 4': '1',
                                       'BI-RADS 5': '1'})
train_df = df[df['split'] == 'training']
tmp_test_df = df[df['split'] == 'test']
test_df, val_df = train_test_split(tmp_test_df, test_size=0.5, stratify=tmp_test_df['label'], random_state=42)


ts_length = len(test_df)
test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))

ts_gen = ImageDataGenerator()
test_gen = ts_gen.flow_from_dataframe(test_df, x_col= 'image_path', y_col= 'label', target_size=(224, 224), class_mode= 'categorical',
                                        color_mode='rgb', shuffle= False, batch_size= test_batch_size)


### **Load model weight**

In [ ]:
# Create Model Structure
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)

base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False,
                                                               weights="imagenet",
                                                               input_shape=img_shape, pooling='max')


model = Sequential([
    base_model,
    BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
    Dense(512, kernel_regularizer=regularizers.l2(l= 0.016),
               activity_regularizer=regularizers.l1(0.006),
               bias_regularizer=regularizers.l1(0.006), activation='relu'),
    Dropout(rate=0.5, seed= 123),
    Dense(2, activation= 'softmax')])

model.load_weights('./output/best/model_weights.h5')

model.compile(Adamax(learning_rate= 0.001),
              loss='categorical_crossentropy',
              metrics= [tf.keras.metrics.FalseNegatives(name='FN'),
                        tf.keras.metrics.Recall(name='recall'),
                        tf.keras.metrics.AUC(curve='PR', name='pr_auc'),
                        f1_metric.F1Score(),
                        'accuracy'])

In [ ]:
preds = model.predict(test_gen)
y_pred = np.argmax(preds, axis=1)
print(y_pred)

In [ ]:
# Save model predction to csv
y_true = test_gen.classes
# Get filenames from the test generator
filenames = test_gen.filenames

# Create a DataFrame with predictions and true labels
results = pd.DataFrame({
    "img_path": filenames,
    "true_class": y_true,
    "predicted": y_pred
})

# Save the DataFrame to a CSV file
results.to_csv("./output/best/prediction_result.csv", index=False)
results

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

print("F1 Score: {}".format(f1_score(y_true,y_pred)))
print("Recall score: {}".format(recall_score(y_true,y_pred)))
print("Precision score: {}".format(precision_score(y_true,y_pred)))

# Extract values from confusion matrix
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

# Calculate Sensitivity and Specificity
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

# Print the results
print(f"Sensitivity (True Positive Rate): {sensitivity:.4f}")
print(f"Specificity (True Negative Rate): {specificity:.4f}")

In [ ]:
g_dict = test_gen.class_indices
classes = list(g_dict.keys())

# Classification report
print(classification_report(test_gen.classes, y_pred, target_names=classes))
scikitplot.metrics.plot_confusion_matrix(y_true, y_pred, normalize=True)

In [ ]:
scikitplot.metrics.plot_confusion_matrix(y_true, y_pred, normalize=False)

In [ ]:
scikitplot.metrics.plot_precision_recall(y_true, preds)